In this notebook, I show the result of training/using the neural network only on the lowest 28 grid points.

In [ ]:
%matplotlib inline

In [ ]:
import xarray as xr
import uwnet
import holoviews as hv
from gnl.xarray import centderiv
from uwnet.thermo import coriolis_ngaqua, ngaqua_y_to_lat

hv.extension('bokeh')
from gnl.colorblind import colorblind
colorblind()

# Short time simulation

In [ ]:
id = "kickass_baekeland"
# id = "elated_leavitt"
def open_id(id):
    ds3d = xr.open_mfdataset(f"../data/samNN/{id}/OUT_3D/*.nc")
    ds2d = xr.open_mfdataset(f"../data/samNN/{id}/OUT_2D/*.nc")

    return xr.merge([ds3d, ds2d])

In [ ]:
ds = open_id(id)

In [ ]:
ds.W[::3,5].plot(col='time', col_wrap=5)

In [ ]:
ds.SLI[::3,5].plot(col='time', col_wrap=5)

In [ ]:
ds.Prec[::6].plot(col='time', col_wrap=5)

In [ ]:
ds.Prec[::5].mean(['x']).plot(x='y', hue='time');

#  Longer simulation

I tried to run a 100 day simulation, but it died after only 6 days

In [ ]:
d2d = xr.open_mfdataset("/Users/noah/Data/0/7a/194be4f48e151e4391a229780b0e69/OUT_2D/*.nc")

In [ ]:
d2d.time.max()

This shows the simulation only lasted about 8 days. I am not sure why it stopped running.

In [ ]:
d2d.Prec[::24].plot(col='time', col_wrap=3)

In [ ]:
d2d.PW[::24].plot(col='time', col_wrap=3)

Eventually the precipitable water field becomes very grainy.

In [ ]:
d2d.LHF[::24].plot(col='time', col_wrap=3)

the other fields are also super grainy.

In [ ]:
d2d.V200[:72:12].plot(col='time', col_wrap=3)

In [ ]:
d2d.U200[::24].plot(col='time', col_wrap=3)

It seems like the model is not dissipative enough. I should run it withhout the neural network to see what happens. The graininess is very clear after 3 days, so that should be long enough.

In [ ]:
d2d.U850[::24].plot(col='time', col_wrap=3)

# Initial value, no physics

```
dopython = .false.
dosgs = .false.
dodamping = .false.
```

In [ ]:
ds = xr.open_mfdataset("../data/samNN/mad_noyce/OUT_2D/*.nc")

In [ ]:
ds.PW[::24].plot(col='time', col_wrap=3)

In [ ]:
ds.V200[::12].plot(col='time', col_wrap=3)

Without the neural network, the fields are not as grainy, but there still seems to be too little dissipation. I will try running the scheme with the active NN and upper atmospheric damping.

# NN on with sponge layer

In [ ]:
ds = xr.open_mfdataset("../data/samNN/prickly_brattain/OUT_2D/*.nc")

Here is the precipitation field:

In [ ]:
ds.Prec[::12].plot(col='time', col_wrap=3)

In [ ]:
ds.V200[::12].plot(col='time', col_wrap=3)

Compared to the older simulation with damping off:

In [ ]:
d2d.V200[:72:12].plot(col='time', col_wrap=3)

The simulation with the sponge layeris slightly less grainy. But the model is still not dissipative enough. Chris has repeatedly mentioned that we will need to parametrize the momentum source. Maybe that would help damp this model adequately. I don't think this is a grid-scale instability because the precipitation field is not diverging too quickly.

# Prognose U and V

I trained a model which prognoses U and V in addition to QT and SLI. Let's load the model and run a single column model simulation for the location x,y = (0, 51), which is in the model midlatitudes.

In [ ]:
path = "/Users/noah/Data/0/8a/13a4068f3166269eef9b0bc1ce0103/1.pkl"
ds = xr.open_dataset("../data/training_data_lower_atmos.nc")
loc = ds.isel(x=slice(0,1), y=slice(50, 51))

mod = uwnet.MLP.from_path(path)
col = uwnet.run_column(mod, loc).squeeze()

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(7,5))


col.U.squeeze().plot(x='time', ax=axs[0,0])
col.UOBS.squeeze().plot(x='time', ax=axs[0,1])
col.V.squeeze().plot(x='time', ax=axs[1,0])
col.VOBS.squeeze().plot(x='time', ax=axs[1,1])
plt.tight_layout()

axs[0,0].set_title("NN")
axs[0,1].set_title("NGAqua")

We can see the predicted wind field, differs pretty dramatically from the training data. Could this be a problem with the forcing data?

In [ ]:
col.FU.plot(x='time')

In [ ]:
col.FV.plot(x='time')

I am a little surprised the large-scale forcings of U and V are so noisy given how coherent the structures in the winds are. What do the forcings look like without any advection contribution? For the $U$ equation lets' look at
$ -p_x + f V $.

In [ ]:

ds = xr.open_dataset("/Users/noah/Data/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/3d/all.nc", chunks={'x': 2, 'y': 2})
rho = xr.open_dataset("/Users/noah/Data/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/stat.nc").RHO[0]


# get coriolis force
fcor = coriolis_ngaqua(ds.y)
px = centderiv(ds.PP, dim='x')/rho
fv = fcor * ds.V

# advection terms
adv = centderiv(ds.U, dim='x') * ds.U + ds.V*centderiv(ds.U, dim='y') + ds.W * centderiv(ds.U, dim='z')

In [ ]:
fv.isel(x=0,y=50).sel(z=col.z).plot(x='time')

In [ ]:
px.isel(x=0,y=50).sel(z=col.z).plot(x='time')

In [ ]:
adv.isel(x=0,y=50).sel(z=col.z).plot(x='time')

We can see that the pressure gradients approximately balance the coriolis force as expected, and that the nonlinear advection also plays an important role. Here is what the sum of all these terms looks like:

In [ ]:
g_U = -px + fv - adv

In [ ]:
g_U.isel(x=0,y=50).sel(z=col.z).plot(x='time')

As we can see, this is similarly noisy as the forcings that were computed using SAM. Therefore, I think the problems with the column performance of horizontal wind budgets probably aren't related to the forcings. Rather, the issue could be that dominant balances in the momentum budget obscure the role of Q3. Perhaps our sampling interval is not short enough to capture the importance of momentum mixing processes?